In [1]:
#!/usr/bin/env python
# Copyright (c) 2020 - The Procedural Generation for Gazebo authors
# For information on the respective copyright owner see the NOTICE file
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
%env GAZEBO_MODEL_PATH=/home/agusmazzeo/.gazebo/models
import os
from tqdm import tqdm
import geopandas as gpd
import argparse
import datetime
from pcg_gazebo.simulation import SimulationModel, \
    add_custom_gazebo_resource_path
from pcg_gazebo.generators.creators import extrude
from pcg_gazebo.generators.shapes import random_rectangles, \
    random_rectangle, random_points_to_triangulation, rectangle
from pcg_gazebo.generators import WorldGenerator
from shapely.geometry import shape, Polygon, MultiPolygon, box as Box, MultiLineString


env: GAZEBO_MODEL_PATH=/home/agusmazzeo/.gazebo/models


In [6]:
wall_thickness = 0.1
wall_height = 5
world_name = "rect_falop"

block_size = 1
# Create a world generator to place
# objects in the world
world_generator = WorldGenerator()
polygons = []
layout = gpd.read_file(f"{os.getcwd()}/files/small_house_clean.dxf")
wall_lines = layout.unary_union
# multi_poly = MultiPolygon(wall_lines)
# Create the wall model based on the extruded
# boundaries of the polygon
walls_model_list = []
for line in tqdm(wall_lines.geoms):  
    box = Box(*line.bounds)
    walls_model_list.append(extrude(
        polygon=line,
        thickness=wall_thickness,
        height=wall_height,
        pose=[box.centroid.x-30, box.centroid.y-10, wall_height / 2., 0, 0, 0],
        extrude_boundaries=False,
        color='xkcd'))
counter = 0
for model in tqdm(walls_model_list):
    counter += 1
    model.name = str(counter) + '_wall'
    world_generator.world.add_model(
    tag=model.name,
    model=model)

print(wall_lines[0])

100%|█████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.35it/s]

LINESTRING (32.92112059049024 37.23766084519904, 26.92112059049024 37.23766084519904, 26.92112059049024 33.88766084519904, 26.52112059049024 33.88766084519904, 26.52112059049024 37.23766084519904, 18.12112059049024 37.23766084519904, 18.12112059049024 33.88766084519904, 17.72112059049024 33.88766084519904, 17.72112059049024 37.23766084519904, 1.321120590490227 37.23766084519904, 1.321120590490227 20.43766084519903, 14.37112059049024 20.43766084519903, 14.37112059049024 20.03766084519904, 13.82112059049024 20.03766084519904, 13.82112059049024 19.43766084519903, 13.42112059049024 19.43766084519903, 13.42112059049024 20.03766084519904, 11.42112059049024 20.03766084519904, 11.42112059049024 16.03766084519905, 13.42112059049024 16.03766084519905, 13.42112059049024 16.63766084519903, 13.82112059049024 16.63766084519903, 13.82112059049024 16.03766084519905, 14.17112059049025 16.03766084519905, 14.17112059049025 15.63766084519903, 7.721120590490227 15.63766084519903, 7.721120590490227 16.03766


/home/agusmazzeo/.local/lib/python3.7/site-packages/ipykernel_launcher.py:33: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.


In [3]:
world_generator.world.add_model(
    tag='ground_plane',
    model=SimulationModel.from_gazebo_model('ground_plane'))

# # Retrieve the free space polygon where objects
# # can be placed within the walls
# free_space_polygon = world_generator.world.get_free_space_polygon(
#     ground_plane_models=[walls_model.name],
#     ignore_models=['ground_plane'])
# # Add the workspace constraint to the
# # generator
# world_generator.add_constraint(
#     name='room_workspace',
#     type='workspace',
#     frame='world',
#     geometry_type='polygon',
#     polygon=free_space_polygon
# )

# # Add constraint to place all object
# # tangent to the ground
# world_generator.add_constraint(
#     name='tangent_to_ground_plane',
#     type='tangent',
#     frame='world',
#     reference=dict(
#         type='plane',
#         args=dict(
#             origin=[0, 0, 0],
#             normal=[0, 0, 1]
#         )
#     )
# )

# orientation_dofs = ['yaw']
# models = dict()
# world_generator.add_asset(
#     tag='box',
#     description=dict(
#         type='box',
#         args=dict(
#             size="__import__('numpy').random.uniform(1, 10, 3)",
#             name='cuboid',
#             color='xkcd'
#         )
#     )
# )
# models['box'] = 120

# # Add placement policy
# placement_policy = dict(
#     models=list(models.keys()),
#     config=[
#         dict(
#             dofs=['x', 'y'],
#             tag='workspace',
#             workspace='room_workspace'
#         ),
#         dict(
#             dofs=orientation_dofs,
#             tag='uniform',
#             mean=0,
#             min=-3.141592653589793,
#             max=3.141592653589793
#         )
#     ]
# )

# # Set local constraints
# local_constraints = list()
# for m in models:
#     local_constraints.append(
#         dict(model=m, constraint='tangent_to_ground_plane'))
# # Place objects randomly on the free
# # space within the walls
# world_generator.add_engine(
#     tag='placement_engine',
#     engine_name='random_pose',
#     models=list(models.keys()),
#     max_num=models,
#     model_picker='random',
#     no_collision=True,
#     min_distance=0.0,
#     policies=[placement_policy],
#     constraints=local_constraints
# )
# # Run placement engine
# world_generator.run_engines(attach_models=True)
    
world_generator.world.name = world_name
world_generator.world.show()

SceneViewer(width=1311, height=713)

In [4]:
import sys
# Export world to file and walls model as Gazebo model
export_models_dir = "/home/agusmazzeo/.gazebo/models"
export_world_dir = "/home/agusmazzeo/.gazebo/worlds"
add_custom_gazebo_resource_path(export_models_dir)
print(sys.getsizeof(world_generator))

full_world_filename = world_generator.export_world(
    output_dir=export_world_dir,
    filename=world_generator.world.name + '.world',
    models_output_dir=export_models_dir,
    with_default_sun=False,
    overwrite=True)

print('World file: {}'.format(full_world_filename))

64
World file: /home/agusmazzeo/.gazebo/worlds/rect_falop.world


In [5]:
%env GAZEBO_MODEL_PATH=/home/agusmazzeo/.gazebo/models

import os

base_dir = os.getcwd()
files_dir = f"{base_dir}/files"

layout_reader = LayoutReader(file_name=f"small_house_clean", file_extension="dxf", files_dir=f"{files_dir}/")
layout_reader.create_gazebo_model(output_file_dir="/home/agusmazzeo/.gazebo", show=True)

env: GAZEBO_MODEL_PATH=/home/agusmazzeo/.gazebo/models


NameError: name 'LayoutReader' is not defined